<a id="1"></a> <br>
# 1. Notebook Preparation

<a id="2"></a> <br>
## 1.1 Libraries Installation

In [1]:
%%capture
%pip install -r requirements.txt

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import scikit_posthocs as sp

<a id="3"></a> <br>
## 1.2 Data Retrieval

In [4]:
cookiecats = pd.read_csv('dataset/cookie_cats.csv')

In [5]:
cookiecats.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [ ]:
columns_to_plot = [col for col in fastfood.columns if col in ['MarketID', 'MarketSize', 'AgeOfStore']]

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(14, 3))
fig.subplots_adjust(hspace=0.3, wspace=0.2)

for i, col in enumerate(columns_to_plot):
    sns.countplot(x=col, data=fastfood, ax=ax[i])

fig.suptitle('Fast Food Data')

plt.show()